# Ticket pricing

You've been shopping around for a holiday package deal and its time to make your choice of which deal to go with. The file [01-holidays.txt](01-holidays.txt) contains a summary of your investigations. 

It's a simple text file, with one possible holiday package per line.

Each line has four fields, separated by spaces. They are:
* The deal ID, from the price comparison website you found it.
* The holiday price, in whole pounds.
* The location of the holiday, always a single word.
* The number of nights you'd be staying. 

For example, the data file might look like this:

```
db61bb90 769 Morgantown 3
202c898b5f 1284 Morgantown 21
def36ffcd 1514 Giessenmestia 21
389018bd0707 1052 Estacada 21
a487c4270 782 Geoje-Si 14
6caf2584a55 724 Stonington-Island 14
199608abc5 1209 Nordkapp 21
```

## Part 1
You have a budget of £1200. How many of the holidays can you afford?

Given the example data above, you could afford four of the holidays: the trips to Estacada, Geoje-Si and Stonnington-Island, and the three-day trip to Morgantown. 

The 21 day trip to Morgantown and the trips to  Giessenmestia and Nordkapp are all too expensive.

# Part 2
You don't just want _a_ holiday. You want the _best_ holiday. What is the code of the holiday which would give you the best value?

The "value" of a holiday is the duration per pound. Because some destinations are better than others, you'll want to scale the value for some locations. For instance, a night in Timbuktu is worth three times as much as a holiday in Bletchley.

Assume all holidays have a relative value of 1, apart from these destinations.

| Destination | Score |
|-------------|-------|
| Almaty | 2.0 |
| Brorfelde | 0.9 |
| Estacada | 0.4 |
| Jayuya | 0.6 |
| Karlukovo | 2.2 |
| Morgantown | 2.9 |
| Nordkapp | 1.5 |
| Nullarbor | 2.2 |
| Puente-Laguna-Garzonkuala-Penyu | 0.4 |
| Uzupis | 0.9 |

## Example

Given the holiday list above, the holiday to Geoje-Si (with the standard weighting of 1) has a value of $\frac{14}{782} = 0.0179$ nights per pound. 

The trip to Estacada looks promising, at $\frac{21}{1052} = 0.0200$ nights per pound. Unfortunately, the weighting for Estacada is low, to the adjusted cost is $0.4 \times \frac{21}{1052} = 0.00798$ nights per pound.

The best value holiday is the 21 day trip to Morgantown, with a value of $2.9 \times \frac{21}{1284} = 0.0474$ nights per pound. Unfortunately, it's unaffordable. 

The best value affordable holiday is the trip to Stonnington Island, with $\frac{14}{1284} = 0.0193$ nights per pound.

# Worked example of solving

For those still getting to grips with things, I thought I'd show how I'd work through solving the day 1 task.

My first step when looking at this kind of thing is to think about data structures and operations. What data do we have, and what do we need to do with it?

We're given a list of holidays. In the first part of the task, we have to filter that list against some condition and count how many meet the condition. In the second part, we have to find the holiday with some maximal value. 

How to store holidays? We just need something we can walk over (iterate over). There are many choices. We could just use a list, or a set (if we assume each holiday is distinct), or a dict/map/hashmap, using the holiday ID as a key. They're all much of a sameness, so let's take the path of least resistance and use a list.

How to represent each holiday? Each holiday is a record of four parts (ID, destination, price, duration). I could use a dict or (in Python) a namedtuple to store the parts. That has the advantage that it gives a name to each of the parts. The alternative is to use a list to store each holiday and refer to the parts by number. Either is good, but as this task is small, I'll be able to get away with just remembering four numbers and what they mean.

So, my chosen representation is a list of lists. That's not the best representation, but it's good enough for this task.

Next step: reading the list of holidays. Let's just start by trying to read in the data file and split it into a list, one element for each holiday. Let's not try to sort out each holiday in that list at the moment.

I could look through the Python documentation for how to do this, or I could hit a search engine. 

I'm not the first to have done that. Typing "python read file lines" suggests and autocomplete of "python read file lines into a list", which is spot on, and leads to a [StackOverflow post](https://stackoverflow.com/questions/3925614/how-do-you-read-a-file-into-a-list-in-python). Result!

Let's try the top-rated answer, with some print statements to check what's going on:
```
with open('01-holidays.txt') as f:
    lines = f.read().splitlines()
print(len(lines), lines[0])
```
That gives output
```
124 dda7d369 1546 Uzupis 21
```
which looks right: 124 lines, and that's the first line in the file.

Another quick search for splitting a string into a list points me in the direction of .split().

I can combine those ideas like this:

```
holidays = []
with open('01-holidays.txt') as f:
    for line in f.read().splitlines():
        holidays += [line.split()]
        
print(len(lines), holidays[0])
```

But I actually like the second answer of that Stack Overflow post better, and prefer to use the list comprehension:

```
holidays = [line.strip().split() for line in open('01-holidays.txt')]

print(len(lines), holidays[0])
```

The basic format is `[<do something to item> for <item> in <bunch of items>]`

I think it's cleaner, and it's certainly more compact.

Now we have the holidays, time to find the affordable ones. 

The basic idea is to keep a count of the number of affordable holidays as we walk along the list of holidays. Every time we get to an affordable one, increase the count by one. The price is in the second element of the holiday record, which is number 1 in Python's count-from-zero world.

That turns quite simply into code:

```
affordable_count = 0
for holiday in holidays:
    if holiday[1] <= 1200:
        affordable_count += 1

print(affordable_count)
```

…which gives an error about unorderable types, comparing `str()` with `int()`.

That makes sense. Strings and numbers are different, and what we've got is a list of strings. A quick search online tells us that `int(something)` will convert `something` into a number, if possible. Let's make that change and try again.

```
affordable_count = 0
for holiday in holidays:
    if int(holiday[1]) <= 1200:
        affordable_count += 1

print(affordable_count)
```

And that gives the right answer!

Now for part 2. 

At first sight, its intimidating. Complex calculation, lots of data to move around. But, let's just start with what we can do easily and see where we end up.

Each destination has a weight for how much it's liked. If a weight's not given, we use 1.

We can store the given weights in a `dict`: it's what they were made for. Associate some value with a key; in this case, the key is the destination name and the value is the weight.

```
destination_values = {'Almaty': 2.0, 'Brorfelde': 0.9, 'Estacada': 0.4, 'Jayuya': 0.6, 'Karlukovo': 2.2, 
                      'Morgantown': 2.9,'Nordkapp': 1.5, 'Nullarbor': 2.2, 
                      'Puente-Laguna-Garzonkuala-Penyu': 0.4, 'Uzupis': 0.9}
```

We can get the values from the dict quite easily:

```
print(destination_values['Jayaya'])
```

How to get the value when the destination isn't in the table? We could use the defaultdict from the Python library, or we could use the `get()` method which accepts a default value, or we could just wrap some control in a function.

```
def value_of_destination(name):
    if name in destination_values:
        return destination_values[name]
    else:
        return 1
```

And try it:

```
print(destination_values['Mamula'])
```

Now we can find the destination weight for each destination, how to find the value of a holiday? Again, wrap that in a function so we can call it without getting bogged down in the details. 

The tricky part here is to keep track of data types. We have to convert strings to numbers again, but division involving integers will often give an integer as the answer. The standard trick in these situation is to make sure that at least one number in each calculation is a floating-point number, and Python will ensure that the whole calculation is done a floating point numbers.

```
def value_of_holiday(holiday):
    hid, cost, destination, duration = tuple(holiday)
    value = value_of_destination(destination) * float(duration) / int(cost)
    return value
```

Again, we can test it

```
print(value_of_holiday(holidays[0]))
```

Now we have the parts for solving the task. Just like part 1, we'll walk over the list of holidays, keeping track of the best value one so far. We have remember two items now, though: the best value, and the code of the holiday with that value. We'll just use two separate variables to keep track of them.

```
best_holiday = ''
best_value = 0

for holiday in holidays:
    if int(holiday[1]) <= 1200:
        if value_of_holiday(holiday) > best_value:
            best_value = value_of_holiday(holiday)
            best_holiday = holiday[0]        

print(best_holiday)
```

Rather than having two `if` statements, we could say 

```
if int(holiday[1]) <= 1200 and value_of_holiday(holiday) > best_value:
```

but I'm not sure it's much better in this context.

And there you have it!

I hope you found the exposition useful.

Yes, I use the internet a lot to look up bits of syntax. It's much the same as looking in a textbook and finding snippets of code to reuse and repurpose. The skill of programming isn't so much the mastery of syntax as it is about understanding the problem and how to put together a solution. By all means look stuff up; a good programmer knows what to look up (or ask) and how to use the answer.

### Solution

In [2]:
holidays = []
with open('01-holidays.txt') as f:
    for hol_line in f:
        holidays.append(hol_line.split())
        
holidays[:3]

[['dda7d369', '1546', 'Uzupis', '21'],
 ['68022753', '1239', 'Mamula', '21'],
 ['b261dbd1cef', '996', 'Holmegaard', '21']]

In [3]:
holidays = [line.split() for line in open('01-holidays.txt')]
        
holidays[:3]

[['dda7d369', '1546', 'Uzupis', '21'],
 ['68022753', '1239', 'Mamula', '21'],
 ['b261dbd1cef', '996', 'Holmegaard', '21']]

In [13]:
affordable_holidays = []
for h in holidays:
    if int(h[1]) <= 1200:
        affordable_holidays.append(h)

len(affordable_holidays)

59

In [5]:
affordable_holidays = [h for h in holidays if int(h[1]) <= 1200]
len(affordable_holidays)

59

In [14]:
len(holidays)

124

### Smart-alec one-line solution

In [4]:
sum(1 for h in open('01-holidays.txt') if int(h.split()[1]) <= 1200)

59

In [16]:
destination_values = {'Almaty': 2.0, 'Brorfelde': 0.9, 'Estacada': 0.4, 'Jayuya': 0.6, 'Karlukovo': 2.2, 
                      'Morgantown': 2.9,'Nordkapp': 1.5, 'Nullarbor': 2.2, 
                      'Puente-Laguna-Garzonkuala-Penyu': 0.4, 'Uzupis': 0.9}

In [17]:
def value_of_destination(name):
    if name in destination_values:
        return destination_values[name]
    else:
        return 1

In [18]:
def value_of_holiday(holiday):
    hid, cost, destination, duration = tuple(holiday)
    value = value_of_destination(destination) * float(duration) / int(cost)
    return value

In [19]:
best_holiday = ''
best_value = 0

for h in affordable_holidays:
    if value_of_holiday(h) > best_value:
        best_value = value_of_holiday(h)
        best_holiday = h[0]
        
best_holiday

'ee064e1e2ea'

## Smart-alec solution

In [20]:
# Right answer
max(affordable_holidays, key=value_of_holiday)[0]

'ee064e1e2ea'

In [21]:
# Answer if you don't filter by affordability
max(holidays, key=value_of_holiday)[0]

'c86e2e5826'

In [23]:
# Answer if you don't scale by perceived value
max(affordable_holidays, key=lambda h: float(h[3]) / float(h[1]))[0]

'f60e203aaaf9'

In [22]:
# Answer if you don't scale by perceived value, AND don't filter by affordability
max(holidays, key=lambda h: float(h[3]) / float(h[1]))[0]

'f60e203aaaf9'